In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()
print('Initalized')

Initalized


# Data Acquisition
Read the case, department, and source data into their own spark dataframes.

In [2]:
case = spark.read.csv("data/case.csv", sep=",", header=True, inferSchema=True).persist()
dept = spark.read.csv("data/dept.csv", sep=",", header=True, inferSchema=True).persist()
source = spark.read.csv("data/source.csv", sep=",", header=True, inferSchema=True).persist()

case.show(1, vertical=True)
dept.show(1, vertical=True)
source.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row

-RECORD 0---------------------------
 sour

Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
Inspect your folder structure. What do you notice?
Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [3]:
source.show(1)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
+---------+----------------+
only showing top 1 row



In [4]:
# source.write.json("source_json", mode="overwrite")
# source.write.json("source_csv", mode="overwrite")

How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [5]:
# rename SLA_due_date to case_due_date
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

# transfoprm case_closed and case_late columns from yes or no to true and false
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn("case_late", expr('case_late == "YES"'))

# turn the council_district column into a string type
case = case.withColumn("council_district", col("council_district").cast("string"))

# transform date columns into date formats
fmt = "M/d/yy H:mm"

case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

# normalize the addresses to all lower case and remove any extra spaces
case = case.withColumn("request_address", trim(lower(case.request_address)))

# show the results vertically
case.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-01 00:46:00  
 case_due_date        | 2018-01-01 00:46:00  
 case_late            | false                
 num_days_late        | -2.0126041

In [6]:
case.filter(case.case_closed == False).select(
    min(case.case_opened_date).alias('oldest_case'),
).select('oldest_case',
        datediff(current_timestamp(), "oldest_case").alias('age_in_days')
).show(1, vertical=True)

-RECORD 0--------------------------
 oldest_case | 2017-01-01 13:48:00 
 age_in_days | 1430                



How many Stray Animal cases are there?

In [7]:
case.createOrReplaceTempView("case")

In [8]:
spark.sql(
"""
Select count(*) from case
where service_request_type = "Stray Animal"
"""
).show()

+--------+
|count(1)|
+--------+
|   26760|
+--------+



How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [9]:
spark.sql(
"""
Select count(*) from case
where dept_division = "Field Operations" and service_request_type != "Officer Standby"
"""
).show()

+--------+
|count(1)|
+--------+
|  113902|
+--------+



Convert the council_district column to a string column.

In [10]:
case = case.withColumn("council_district", col("council_district").cast("string"))
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
only showing top 1 row




Extract the year from the case_closed_date column.

In [11]:
case = case.withColumn("closed_year", year("case_closed_date"))
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 closed_year          | 2018                 
only showing top 1 row



Convert num_days_late from days to hours in new columns num_hours_late.

In [12]:
case = case.withColumn("num_hours_late", col("num_days_late") * 24)
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 closed_year          | 2018                 
 num_hours_late       | -23964.2102784       
only showing top 1 row



Join the case data with the source and department data.

In [13]:
source.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [14]:
joined = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
    .join(source, "source_id", "left")
    .drop(source.source_id)
)

joined.show(1, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 closed_year          | 2018                 
 num_hours_late       | -23964.2102784       
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | svcCRMLS             
only showing top 1 row



Are there any cases that do not have a request source?

In [15]:
joined.createOrReplaceTempView("joined")

In [16]:
spark.sql(
"""
select count(*) from joined
where service_request_type is null
"""
).show(1)

+--------+
|count(1)|
+--------+
|       0|
+--------+



What are the top 10 service request types in terms of number of requests?

In [17]:
spark.sql(
"""
select service_request_type, count(*) as number_of_request
from joined
group by service_request_type
order by number_of_request desc
"""
).show(10)

+--------------------+-----------------+
|service_request_type|number_of_request|
+--------------------+-----------------+
|           No Pickup|            89210|
|Overgrown Yard/Trash|            66403|
|        Bandit Signs|            32968|
|        Damaged Cart|            31163|
|Front Or Side Yar...|            28920|
|        Stray Animal|            27361|
|Aggressive Animal...|            25492|
|Cart Exchange Req...|            22608|
|Junk Vehicle On P...|            21649|
|     Pot Hole Repair|            20827|
+--------------------+-----------------+
only showing top 10 rows



* What are the top 10 service request types in terms of average days late?
* Does number of days late depend on department?
  * No, although DSD/Code Enforcment appears to have the most delayed cases for the top 10 issues
* How do number of days late depend on department and request type?
  * Appears that the top 10 worst performing case clear are related speifically to the service type and not the department that handles them

In [18]:
spark.sql(
"""
select service_request_type, avg(num_days_late) as avg_days_late, count(*) as number_of_cases
from joined
group by service_request_type
order by avg_days_late desc
"""
).show(10)

+--------------------+------------------+---------------+
|service_request_type|     avg_days_late|number_of_cases|
+--------------------+------------------+---------------+
|  Zoning: Junk Yards| 175.9563621042095|            296|
|Labeling for Used...|162.43032902285717|              7|
|Record Keeping of...|153.99724039428568|              7|
|Signage Requied f...|151.63868055333333|             12|
|Storage of Used M...|142.11255641500003|              8|
|Zoning: Recycle Yard|135.92851612479797|            198|
|Donation Containe...|131.75610506358709|            155|
|License Requied U...|128.79828704142858|              7|
|Traffic Signal Gr...|101.79846062200002|              5|
|           Complaint| 72.87050230311691|           2420|
+--------------------+------------------+---------------+
only showing top 10 rows



In [19]:
spark.sql(
"""
select service_request_type, department, avg(num_days_late) as avg_days_late, count(*) as number_of_cases 
from joined
group by service_request_type, department
order by avg_days_late desc
"""
).show(10)

+--------------------+--------------------+------------------+---------------+
|service_request_type|          department|     avg_days_late|number_of_cases|
+--------------------+--------------------+------------------+---------------+
|  Zoning: Junk Yards|DSD/Code Enforcement| 175.9563621042095|            296|
|Labeling for Used...|DSD/Code Enforcement|162.43032902285717|              7|
|Record Keeping of...|DSD/Code Enforcement|153.99724039428568|              7|
|Signage Requied f...|DSD/Code Enforcement|151.63868055333333|             12|
|Storage of Used M...|DSD/Code Enforcement|142.11255641500003|              8|
|Zoning: Recycle Yard|DSD/Code Enforcement|135.92851612479797|            198|
|Donation Containe...|DSD/Code Enforcement|131.75610506358709|            155|
|License Requied U...|DSD/Code Enforcement|128.79828704142858|              7|
|Traffic Signal Gr...|Trans & Cap Impro...|101.79846062200002|              5|
|           Complaint|    Customer Service| 72.87050

In [20]:
spark.sql(
"""
select department, avg(num_days_late) as avg_days_late, count(*) as number_of_cases 
from joined
group by department
order by avg_days_late desc
"""
).show(10)

+--------------------+-------------------+---------------+
|          department|      avg_days_late|number_of_cases|
+--------------------+-------------------+---------------+
|    Customer Service| 59.737091496300785|           2889|
|         Solid Waste| -2.200057513672161|         286287|
|        Metro Health| -4.911766979607001|           5313|
|  Parks & Recreation| -5.251521960055153|          19964|
|Trans & Cap Impro...| -20.61283735405272|          97841|
|DSD/Code Enforcement| -38.36938892614484|         323579|
|Animal Care Services|-226.51783940550325|         119362|
|        City Council|               null|             34|
+--------------------+-------------------+---------------+



In [21]:
joined.write.csv("joined_csv", mode="overwrite")

In [22]:
joined.show(5, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 closed_year          | 2018                 
 num_hours_late       | -23964.2102784       
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | svcCRMLS             
-RECORD 1------------------------------------
 source_id            | svcCRMSS             
 case_id              | 1014127333